## Configure a face library system

 Structure of our library system:

```library```<br/>
|__```celebrity_1``` <br/>
|   |   meta.npz
|    |   face_1.png
|    |   face_2.png
|    
|__```celeberity_2```
|
|
|
|___```(videoname)_(cluster no.)```
    

In [1]:
# Constants here
LIB_ROOT = './library' #'/home/ubuntu/visual_search_assistant/M3/library'
META_FNAME = 'meta.npz'

### Pre-processing the library
Extract encoding from faces

In [2]:
import face_recognition
import os
import numpy as np

def extract_encodings_dir(dir,mean=False):
    images = []
    names = []
    encodings = []
    
    for image_pth in os.listdir(dir):
        if image_pth.endswith('npz'):
            continue
        img = face_recognition.load_image_file(os.path.join(dir,image_pth))
        images.append(img)
        names.append(image_pth)
    
    for idx,(img,name) in enumerate(zip(images,names)):
        print('[%d / %d] Extracting encoding for %s' % (idx+1,len(images),name))
        embd = face_recognition.face_encodings(img)[0]
        encodings.append(embd)
    
    if mean:
        res = np.mean(encodings,axis=0)
        return res
    else:
        return encodings
    
    

### Pre-process the library

In [9]:
import os
import pickle

# preprocessing : convert single files into directories
identities = os.listdir(LIB_ROOT)
metadata_pth = os.path.join(LIB_ROOT,'meta.npz')
metadata = {
}
for fname in identities:
    src_pth = os.path.join(LIB_ROOT,fname)
    
    if not os.path.isdir(src_pth):
        name = fname.split('.')[0]
        print(name)
        dst_pth = os.path.join(LIB_ROOT,name)
        os.mkdir(dst_pth) # create directory for the identity
        os.rename(src_pth,os.path.join(dst_pth,fname)) # move single image file
        src_pth = dst_pth
        print("Created new directory for %s" % name)
    else:
        name = fname
        
    mean_encoding = extract_encodings_dir(src_pth,mean=True)
    metadata[fname] = {
        "encoding":mean_encoding,
        "name":'',
        "id":fname,
        "clusterlist":[fname]
    }
    
    print("Processing identity %s" % name)
with open(metadata_pth,'wb') as f:
    pickle.dump(metadata,f,protocol=pickle.HIGHEST_PROTOCOL)

[1 / 1] Extracting encoding for jongshin_yoon.jpg
Processing identity jongshin_yoon
hangjoon_jang
Created new directory for hangjoon_jang
[1 / 1] Extracting encoding for hangjoon_jang.jpeg
Processing identity hangjoon_jang
Processing identity .ipynb_checkpoints
Donna_Meagle
Created new directory for Donna_Meagle
[1 / 1] Extracting encoding for Donna_Meagle.jpg


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Processing identity Donna_Meagle
[1 / 1] Extracting encoding for gookjin_kim.jpeg
Processing identity gookjin_kim
[1 / 1] Extracting encoding for leslie_knope.jpg
Processing identity leslie_knope
meta
Created new directory for meta
Processing identity meta
[1 / 1] Extracting encoding for goora_kim.jpg
Processing identity goora_kim


Check if correctly saved

In [11]:
example_meta_pth = './library/meta.npz' #'/home/ubuntu/visual_search_assistant/M3/library//meta.npz'
import pickle

with open(example_meta_pth,'rb') as rf:
    metadata = pickle.load(rf)

print(metadata.keys())
print(metadata['goora_kim'].keys())

dict_keys(['jongshin_yoon', 'hangjoon_jang.jpeg', '.ipynb_checkpoints', 'Donna_Meagle.jpg', 'gookjin_kim', 'leslie_knope', 'meta.npz', 'goora_kim'])
dict_keys(['encoding', 'name', 'id', 'clusterlist'])
